In [1]:
#CSV Bibliothek importieren 
import csv
#Pandas importieren und als pd bezeichnen
import pandas as pd
#Numpy importieren und als np bezeichnen 
import numpy as np
#Json Biliothek importieren
import json
#Obere Schranke für die Größe der Daten auf 100,000,000 setzen  
csv.field_size_limit(100000000)
#Maximale Anzahl von Zeilen und Spalten Zeigen 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
#Die Warnings zu Konsole leiten
import warnings
warnings.filterwarnings('ignore')
#Pydomain-Package importieren
from pydomains import *

---

# Kategorisierung der Domäne 
#### Zuerst werden die Tweets in TweetsCov19 bearbeitet, dann die Tweets in TweetsKB

1. Daten werden gelesen.
2. Die einzigartige Domäne werden extrahiert.
3. Die extrahierten Domäne werden ktegorisiert.(Falls Domain zu mehrere Kategorien zugewiesen wurde, wird die Erste ausgewählt)
4. Anhand der kategorisierten Domäne, werden alle anderen Domäne in dem Datensazu kategorisieret.
- Der Grund für diesen Ansatz ist:<br>
    1. Die Geschwindigkeit zu erhöhen, da das Modell langsam ist. Es könnte nicht für alle Tweets immer wieder geladen werden.
    2. Es existieren viele Domain-Wiederholungen im Datensatz
    3. Der Ansatz wurde für jeden Datensatz separat angewendet


***

**TweetsCov19 :**
- Dataframe lesen
- Die einzigartige Domäne aussortieren

In [4]:
#Labels und Pfad
TweetsCov19_labels = ['Tweet_Id', "Benutzername", 'Zeitstempel', 'Abonnenten', 'Abonnierten', 'Retweets', 'Likes', 'Entities', 'Stimmung', 'Erwähnungen', 'Hashtags', 'URLs', 'Text', 'Ist_gelöscht', 'Anzahl der Erwähnungen', 'Anzahl der Hashtags', 'Anzahl der URLs', 'Domäne', 'Textlänge', 'Lesbarkeit']
TweetsCov19_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/tweetscov19_sample/TweetsCov19_stichprobe_lesbarkeit.tsv'

In [5]:
#Stichprobe von Tweetscov19 lesen

df_tweetscov19 = pd.read_csv(TweetsCov19_Pfad, sep='\t', error_bad_lines=False, names=TweetsCov19_labels)
print('Form:', df_tweetscov19.shape)
df_tweetscov19.head()

Form: (9409841, 20)


,Tweet_Id,Benutzername,Zeitstempel,Abonnenten,Abonnierten,Retweets,Likes,Entities,Stimmung,Erwähnungen,Hashtags,URLs,Text,Ist_gelöscht,Anzahl der Erwähnungen,Anzahl der Hashtags,Anzahl der URLs,Domäne,Textlänge,Lesbarkeit
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,2019-09-30 22:00:01+00:00,764191,132,19,180,foodie:Foodie:-2.096663959988218;,1 -1,null;,HalloweenTime?,null;,Has your appetite materialized just as fast as...,True,0,1,0,0,155,76.553
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,2019-09-30 22:00:02+00:00,321189,26833,32,43,mainland china:Mainland_China:-1.6252666161640...,1 -1,null;,HongKong,https://www.catholicnewsagency.com/news/amid-h...,The persecution of Catholics in mainland China...,False,0,1,1,catholicnewsagency.com,204,29.468
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,2019-09-30 22:00:03+00:00,14209,10623,90,182,brexit:Brexit:-1.582809637810362;hear hear:Hea...,1 -2,null;,Brexit,null;,"Hear, hear..\nEmily: You were an outspoken cri...",False,0,1,0,0,163,79.534
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,2019-09-30 22:00:03+00:00,2264986,655,60,528,null;,3 -1,amstandardca.,BlueJays,null;,We wrapped up the season with a WINNING week! ...,False,1,1,0,0,132,85.889
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,2019-09-30 22:00:03+00:00,171,138,33,117,my family:My_Family:-2.6353832182503094;roll t...,4 -1,null;,null;,null;,Super excited to announce that I will be spend...,False,0,0,0,0,257,61.203


In [11]:
#Zuerst wird hier eine neue Dataframe erzeugt, die nur die Domäne enthält.
#Dann geht die funktion jede zeile der Dataframe durch, und alle Domäne, die in einem Tweet geben, durchsucht.
#Jede neue Domain wird zu der Dictionary hinzugefügt.
#Bei jeder Wiederholung erhöht die Methode die Value von dementsprechender Domain um eins.

df_tweetsCov19_domaene = df_tweetscov19[['Domäne']]

dict_tweetscov19_domaene = {}

for idx, row in df_tweetsCov19_domaene.iterrows():
    list_domaene = []
    list_domaene = (row.values.tolist()[0])
    list_domaene = str(list_domaene).split()
    
    list_domaene = np.array(list_domaene)
    list_domaene = np.unique(list_domaene)
    list_domaene = list_domaene.tolist()
    
    for i in range(0, len(list_domaene)):
        domain = list_domaene[i]
        if domain not in dict_tweetscov19_domaene:
            dict_tweetscov19_domaene[domain] = 1
        else :
            dict_tweetscov19_domaene[domain] = dict_tweetscov19_domaene[domain] +1
    #print(dict_tweetscov19_domaene)

In [12]:
#Hier wird aus Dictionary eine Dataframe erzeugt
df_dict_tweetsCov19_domaene = pd.DataFrame(dict_tweetscov19_domaene.items(), columns=['Domain','Zahl'])

In [13]:
anzahl_der_domaene = df_tweetscov19['Anzahl der URLs'].sum()
print('Anzahl der benutzten Domäne für TweetsCov19(unique) :', df_dict_tweetsCov19_domaene.Domain.size)
print('Anzahl der benutzten Domäne für TweetsCov19(sum) :', anzahl_der_domaene)

Anzahl der benutzten Domäne für TweetsCov19(unique) : 181023
Anzahl der benutzten Domäne für TweetsCov19(sum) : 2508542


In [14]:
#Zuerst wird den prozentualen Ansatz der Domäne im Datensatz berechnet.
#Dann wird die Datafarme aufsteigend sortiert und einen Teil davon wird ausgegeben.
df_dict_tweetsCov19_domaene['prozent'] =  df_dict_tweetsCov19_domaene['Zahl'].apply(lambda row : ((row /anzahl_der_domaene) * 100) )

df_dict_tweetsCov19_domaene['Zahl'] = df_dict_tweetsCov19_domaene['Zahl'].astype(int) 
df_dict_tweetsCov19_domaene.sort_values(by = ['Zahl'], ascending = False, inplace = True)
df_dict_tweetsCov19_domaene.head(10)

,Domain,Zahl,prozent
0,0,6990537,278.669323
4,twitter.com,268243,10.693184
19,youtube.com,119790,4.775284
52,instagram.com,60505,2.411959
8,nytimes.com,35565,1.417756
89,theguardian.com,33835,1.348791
11,cnn.com,29813,1.188459
25,washingtonpost.com,25053,0.998708
136,reuters.com,21106,0.841365
110,facebook.com,20472,0.816092


In [15]:
#Domäne speichern.
pfad_tsv2 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_sample/TweetsCov19_stichprobe_domaene.tsv'
with open(pfad_tsv2, 'w', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        for idx, row in df_dict_tweetsCov19_domaene.iterrows():
            tsv_writer.writerow(row.values.tolist())

***

- Test von Pydomain:

In [2]:
df = pd.DataFrame([{'url': 'bbc.com'}])
df_shalla_test = shalla_cat(df, domain_names='url')

Using cached Shallalist data from local (C:\Users\mdsam\.pydomains\shalla_2017.csv.bz2)...
Loading Shallalist data file...


In [3]:
df_shalla_test

,url,shalla_2017_domain,shalla_2017_cat
0,bbc.com,bbc.com,news


In [4]:
df_shalla_test.values.tolist()[0]

['bbc.com', 'bbc.com', 'news']

<br>

**Die einzigartigen Domäne kategorisieren**

***

In [16]:
#Hier wird mit Hilfe von Pydomain die Domäne kategorisiert
df_tweetsCov19_domaene = df_dict_tweetsCov19_domaene[['Domain']]
df_tweetscov19_domaaene_shalla_kat = shalla_cat(df_tweetsCov19_domaene, domain_names='Domain')

Using cached Shallalist data from local (C:\Users\mdsam\.pydomains\shalla_2017.csv.bz2)...
Loading Shallalist data file...


In [17]:
df_tweetscov19_domaaene_shalla_kat.head()

,Domain,shalla_2017_domain,shalla_2017_cat
0,0,0,NaN
1,twitter.com,twitter.com,socialnet
2,youtube.com,youtube.com,movies
3,instagram.com,instagram.com,socialnet|imagehosting
4,nytimes.com,nytimes.com,news


In [20]:
#Erste Zeile wird gelöscht.
df_tweetscov19_domaaene_shalla_kat.drop(index=df_tweetskb_domaaene_shalla_kat.index[:1], axis=0, inplace=True)

In [21]:
#Eine Spalte der Dataframe wird gelöscht.
#Dataframe wird neu indexiert.
df_tweetscov19_domaaene_shalla_kat = df_tweetscov19_domaaene_shalla_kat[['Domain', 'shalla_2017_cat']]
df_tweetscov19_domaaene_shalla_kat.head()

,Domain,shalla_2017_cat
1,twitter.com,socialnet
2,youtube.com,movies
3,instagram.com,socialnet|imagehosting
4,nytimes.com,news
5,theguardian.com,news


In [22]:
#Hier wird mithilfe von get_cat() die erste Kategorie für die Domain ausgewäht, die zu mehrere Kategorien zugewiesen wurde.
def get_cat(domains):
    l = domains.split('|')
    return l[0]

df_tweetscov19_domaaene_shalla_kat['shalla_2017_cat'] = df_tweetscov19_domaaene_shalla_kat['shalla_2017_cat'].astype(str)

df_tweetscov19_domaaene_shalla_kat['dom_cat'] = df_tweetscov19_domaaene_shalla_kat.apply(lambda row: get_cat(row['shalla_2017_cat']), axis=1)
print('Domains Kategorisiert!')


Domains Kategorisiert!


In [24]:
df_tweetscov19_domaaene_shalla_kat.head()

,Domain,shalla_2017_cat,dom_cat
1,twitter.com,socialnet,socialnet
2,youtube.com,movies,movies
3,instagram.com,socialnet|imagehosting,socialnet
4,nytimes.com,news,news
5,theguardian.com,news,news


In [25]:
#Kategorisierten Daten speichern.
pfad_tsv2 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_sample/TweetsCov19_stichprobe_domaene_kategorisiert.tsv'

with open(pfad_tsv2, 'w', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        for idx, row in df_tweetscov19_domaaene_shalla_kat.iterrows():
            tsv_writer.writerow(row.values.tolist())

***

**Alle Domäne in Tweets mithilfe der Domain-Kategorie-Liste kategorisieren** 

***

In [26]:
#Hir wird jede Zeile der Datframe(ein Tweet) gelesen, die benutzten Domäne werden zu get_dom_categorisiert() geschickt.
#Für jede Domain in Tweet such die Methode die Liste.
#Falls die Domain kategorisiert ist, nimmt sie die Kategorie von der Domain.
#Alle benutzten Kategorien werden mit einer Leerzeichen dazwischen zusammengefügt.
#Das Endergebnis wird zurückgegeben.

def get_dom_categorisiert(domains):
    dom = ''
    list_dom = domains.split()
    for i in range(0, len(list_dom)):
        line = df_tweetscov19_domaaene_shalla_kat.loc[df_tweetscov19_domaaene_shalla_kat['Domain'] == list_dom[i]]
        line = line.values.tolist()
        #print(line)
        try :
            if i == 0 :
                dom = dom + line[0][2]
            else:
                dom = dom + ' ' +line[0][2]
        except IndexError:
            print('Index out of range!')
            #print(line)
            
    return dom

df_tweetscov19['Domäne'] = df_tweetscov19['Domäne'].astype(str)

df_tweetscov19['dom_cat'] = df_tweetscov19.apply(lambda row: get_dom_categorisiert(row['Domäne']) if (row['Domäne'] != '0') else 0, axis=1)
print('Domains!!!')



Domains!!!


In [14]:
#Dataframe wird eingeschränkt und gespeichert.
df_tweetscov19 = df_tweetscov19[['Tweet_Id', 'Ist_gelöscht', 'Domäne', 'Anzahl der URLs', 'dom_cat']]

pfad2='C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/urls/predicted/shalla/tweetscov19_domains_shalla_categorisieret.tsv'

with open(pfad2, 'w', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        for idx, row in df_tweetscov19.iterrows():
            tsv_writer.writerow(row.values.tolist())

***

**Die Anzahl der benutzten Domäne, die zu 10 meist benutzten Kategorien gehören, berechnen**

****

In [30]:
#Hier wird die kategorisierten Domäne wieder gelesen. 
pfad ='C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/urls/predicted/shalla/tweetscov19_domains_shalla_categorisieret.tsv'

df_tweetscov19_domains_shalla_cat = pd.read_csv(pfad, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Ist_gelöscht', 'Domäne', 'Anzahl der URLs', 'dom_cat'])
print(df_tweetscov19_domains_shalla_cat.shape)
df_tweetscov19_domains_shalla_cat.head()

(9409841, 5)


,Tweet_Id,Ist_gelöscht,Domäne,Anzahl der URLs,dom_cat
0,1178791787386814465,False,0,0,0
1,1178791985106153472,True,twitter.com,1,socialnet
2,1178793230223183872,False,goo.gl,1,urlshortener
3,1178795172206919680,True,0,0,0
4,1178798309491822592,False,0,0,0


In [31]:
#Hier werden wie davor die meist benutzten Kategorien berechnet.
df_tweetscov19_domains_shalla_cat['dom_cat'] = df_tweetscov19_domains_shalla_cat['dom_cat'].astype(str)

dict_tweetscov_domains_cat = {}
x = 0
list_cat = []
for idx, row in df_tweetscov19_domains_shalla_cat.iterrows():
    list_cat = (row.values.tolist()[4])
    list_cat = list_cat.split()
    for i in range(0, len(list_cat)):
        cat = list_cat[i]
        x += 1
        if cat not in dict_tweetscov_domains_cat:
            dict_tweetscov_domains_cat[cat] = 1
        else :
            dict_tweetscov_domains_cat[cat] = dict_tweetscov_domains_cat[cat] +1
    #if x == 50:
    #    break

In [32]:
#Die erzeugte Dictionary wird zu einer Dataframe umgewandelt.
df_dict_tweetscov19_dom_cat = pd.DataFrame(dict_tweetscov_domains_cat.items(), columns=['Domain_kat','Zahl'])

In [33]:
print('Anzahl der Kategorien:' ,df_dict_tweetscov19_dom_cat.Domain_kat.size)

Anzahl der Kategorien: 75


In [34]:
URLs_sum = df_tweetscov19_domains_shalla_cat['Anzahl der URLs'].sum()

In [35]:
#Hier wird die prozentualen Anteil der Kategorien im Datensatz berechnet, sortiert und ausgegeben.
df_dict_tweetscov19_dom_cat['prozent'] =  df_dict_tweetscov19_dom_cat['Zahl'].apply(lambda row : ((row /URLs_sum) * 100) )

df_dict_tweetscov19_dom_cat['Zahl'] = df_dict_tweetscov19_dom_cat['Zahl'].astype(int) 
df_dict_tweetscov19_dom_cat.sort_values(by = ['Zahl'], ascending = False, inplace = True)
df_dict_tweetscov19_dom_cat = df_dict_tweetscov19_dom_cat.reset_index()
df_dict_tweetscov19_dom_cat.head(15)

,index,Domain_kat,Zahl,prozent
0,0,0,6990537,278.669323
1,5,nan,1126830,44.919718
2,4,news,641385,25.568039
3,1,socialnet,364531,14.531588
4,3,movies,125938,5.020366
5,11,searchengines,37265,1.485524
6,7,shopping,27715,1.104825
7,13,radiotv,23925,0.953741
8,8,forum,20242,0.806923
9,16,music,19372,0.772241


In [10]:
#Die Anzahl der benutzten Domäne, die zu Kategorie News gehören wird hier berechnet
def get_news_num(dom_cat):
    l = dom_cat.split()
    num = l.count('news')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['dom_cat'] = df_tweetscov19_domains_shalla_cat['dom_cat'].astype(str) 

df_tweetscov19_domains_shalla_cat['news'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_news_num(row['dom_cat']), axis=1)
print('news!')

news!


In [14]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Soziale Netzwerke gehören wird hier berechnet
def get_socialnet_num(dom_cat):
    l = dom_cat.split()
    num = l.count('socialnet')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['socialnet'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_socialnet_num(row['dom_cat']), axis=1)
print('socialnet!')

socialnet!


In [15]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Movies gehören wird hier berechnet
def get_movies_num(dom_cat):
    l = dom_cat.split()
    num = l.count('movies')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['movies'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_movies_num(row['dom_cat']), axis=1)
print('movies!')

movies!


In [16]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Searchengines gehören wird hier berechnet
def get_searchengines_num(dom_cat):
    l = dom_cat.split()
    num = l.count('searchengines')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['searchengines'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_searchengines_num(row['dom_cat']), axis=1)
print('searchengines!')

searchengines!


In [17]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Shopping gehören wird hier berechnet
def get_shopping_num(dom_cat):
    l = dom_cat.split()
    num = l.count('shopping')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['shopping'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_shopping_num(row['dom_cat']), axis=1)
print('shopping!')

shopping!


In [18]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Radiotv gehören wird hier berechnet
def get_radiotv_num(dom_cat):
    l = dom_cat.split()
    num = l.count('radiotv')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['radiotv'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_radiotv_num(row['dom_cat']), axis=1)
print('radiotv!')

radiotv!


In [19]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Forum gehören wird hier berechnet
def get_forum_num(dom_cat):
    l = dom_cat.split()
    num = l.count('forum')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['forum'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_forum_num(row['dom_cat']), axis=1)
print('forum!')

forum!


In [20]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Music gehören wird hier berechnet
def get_music_num(dom_cat):
    l = dom_cat.split()
    num = l.count('music')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['music'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_music_num(row['dom_cat']), axis=1)
print('music!')

music!


In [21]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Porn gehören wird hier berechnet
def get_porn_num(dom_cat):
    l = dom_cat.split()
    num = l.count('porn')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['porn'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_porn_num(row['dom_cat']), axis=1)
print('porn!')

porn!


In [22]:
#Die Anzahl der benutzten Domäne, die zu Kategorie goverment gehören wird hier berechnet
def get_government_num(dom_cat):
    l = dom_cat.split()
    num = l.count('government')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['government'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_government_num(row['dom_cat']), axis=1)
print('government!')

government!


In [23]:
#Die Anzahl der benutzten Domäne, die alles außer bis jetzt bestimmten Kategorien sind, wird hier berechnet
def get_others_num(dom_cat):
    l = dom_cat.split()
    num = l.count('news') + l.count('socialnet') + l.count('movies') + l.count('searchengines') + l.count('shopping') + l.count('radiotv') + l.count('forum') + l.count('music') + l.count('porn') + l.count('government') + l.count('0') + l.count('nan')
    num =  len(l) - num
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetscov19_domains_shalla_cat['others'] = df_tweetscov19_domains_shalla_cat.apply(lambda row: get_others_num(row['dom_cat']), axis=1)
print('others!')

others!


In [28]:
#Die Ergebnisse speichern.
pfad2='C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/urls/predicted/shalla/tweetscov19_domains_shalla_categorisieret_top_ten.tsv'

with open(pfad2, 'w', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        for idx, row in df_tweetscov19_domains_shalla_cat.iterrows():
            tsv_writer.writerow(row.values.tolist())

***

**TweetsKB:**
- Dataframe lesen
- Die einzigartige Domäne aussortieren

In [2]:
#Labels und Pfad
TweetsKB_labels    = ['Tweet_Id', 'Benutzername', 'Zeitstempel', 'Abonnenten', 'Abonnierten', 'Retweets', 'Likes', 'Text', 'URLs', 'Ist_gelöscht', 'Hashtags', 'Erwähnungen', 'Anzahl der Hashtags', 'Anzahl der Mentions', 'Textlänge', 'Lesbarkeit', 'Domäne', 'Anzahl der URLs', 'vader pos', 'vader neu', 'vader neg', 'vader mittelwert' ]
TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/random_sample/emoji/tweetskb_vader.tsv'

In [3]:
#TweetsKB wird gelesen
df_tweetskb = pd.read_csv(TweetsKB_Pfad, sep='\t', error_bad_lines=False, names=TweetsKB_labels)
print('Form :', df_tweetskb.shape)
df_tweetskb.head()

Form : (9409841, 22)


,Tweet_Id,Benutzername,Zeitstempel,Abonnenten,Abonnierten,Retweets,Likes,Text,URLs,Ist_gelöscht,Hashtags,Erwähnungen,Anzahl der Hashtags,Anzahl der Mentions,Textlänge,Lesbarkeit,Domäne,Anzahl der URLs,vader pos,vader neu,vader neg,vader mittelwert
0,1178791636039589889,DCComics,2019-09-30 22:00:01+00:00,3349606,563,2,13,"Today on #DCDaily, the crew takes a deep dive ...",http://bit.ly/2mlNYpS:-:,False,['DCDaily'],['DCUTitans'],1,1,192,70.350,dcuniverseinfinite.com,1,0.000,1.000,-0.0,0.0000
1,1178791641265639424,seanberdyz,2019-09-30 22:00:02+00:00,1203,499,4,20,𝘪’𝘮 𝘨𝘰𝘯𝘯𝘢 𝘣𝘦 𝘳𝘪𝘨𝘩𝘵 𝘩𝘦𝘳𝘦 ⟶ https://t.co/kOxWQL5...,http://ourownside.carrd.co:-:,True,[],[],0,0,73,82.390,carrd.co,1,0.000,1.000,-0.0,0.0000
2,1178791640573579264,AnArtistAtBirth,2019-09-30 22:00:02+00:00,109,215,0,0,❛ 𝘠𝘰𝘶'𝘳𝘦 𝘴𝘪𝘵𝘵𝘪𝘯' 𝘰𝘯 𝘺𝘰𝘶𝘳 𝘧𝘦𝘦𝘭𝘪𝘯𝘨𝘴 . . . 𝘐'𝘮 𝘴𝘪...,null;,True,['𝘛𝘩𝘦𝘰𝘯𝘦'],[],1,0,128,123.755,0,0,0.000,1.000,-0.0,0.0000
3,1178791644797255680,lagosboygang,2019-09-30 22:00:03+00:00,571,1401,0,0,@Tiffany_o_ @UG_derrick That's the one i wanna...,null;,True,[],"['Tiffany_o_', 'UG_derrick']",0,2,77,105.921,0,0,0.000,1.000,-0.0,0.0000
4,1178791648056217600,Phenom_Hoops,2019-09-30 22:00:04+00:00,25630,1674,1,1,2022 6’9 Perry Smith Jr. showcasing his high-m...,https://www.phenomhoopreport.com/2022-69-perry...,False,['PhenomHoops'],[],1,0,227,65.943,phenomhoopreport.com,1,0.058,0.942,-0.0,0.2263


In [7]:
#Zuerst wird hier eine neue Dataframe erzeugt, die nur die Domäne enthält.
#Dann geht die funktion jede zeile der Dataframe durch, und alle Domäne, die in einem Tweet geben, durchsucht.
#Jede neue Domain wird zu der Dictionary hinzugefügt.
#Bei jeder Wiederholung erhöht die Methode die Value von dementsprechender Domain um Eins. 

df_tweetskb_domaene = df_tweetskb[['Domäne']]
dict_tweetskb_domaene = {}

for idx, row in df_tweetskb_domaene.iterrows():
    list_domane = []
    list_domane = (row.values.tolist()[0])
    list_domane = str(list_domane).split()
    
    list_domane = np.array(list_domane)
    list_domane = np.unique(list_domane)
    list_domane = list_domane.tolist()
    
    for i in range(0, len(list_domane)):
        domain = list_domane[i]
        if domain not in dict_tweetskb_domaene:
            dict_tweetskb_domaene[domain] = 1
        else :
            dict_tweetskb_domaene[domain] = dict_tweetskb_domaene[domain] +1
    #print(dict_tweetskb_domaene)

In [8]:
#Hier wird aus Dictionary eine Dataframe erzeugt
df_dict_tweetskb_domaene = pd.DataFrame(dict_tweetskb_domaene.items(), columns=['Domain','Zahl'])

In [9]:
domaene_sum = df_tweetskb['Anzahl der URLs'].sum()
print('Anzahl der benutzten Domains für Tweetskb(unique) :', df_dict_tweetskb_domaene.Domain.size)
print('Anzahl der benutzten Domains für Tweetskb(sum) :', domaene_sum)

Anzahl der benutzten Domains für Tweetskb(unique) : 147069
Anzahl der benutzten Domains für Tweetskb(sum) : 1460380


In [10]:
#Zuerst wird den prozentualen Ansatz der Domäne im Datensatz berechnet.
df_dict_tweetskb_domaene['prozent'] =  df_dict_tweetskb_domaene['Zahl'].apply(lambda row : ((row /domaene_sum) * 100) )
df_dict_tweetskb_domaene.head()

,Domain,Zahl,prozent
0,dcuniverseinfinite.com,4,0.000274
1,carrd.co,479,0.032800
2,0,8024014,549.446993
3,phenomhoopreport.com,9,0.000616
4,airtime.pro,909,0.062244


In [11]:
#Die Datafarme wird aufsteigend sortiert und einen Teil davon wird ausgegeben.
df_dict_tweetskb_domaene['Zahl'] = df_dict_tweetskb_domaene['Zahl'].astype(int) 
df_dict_tweetskb_domaene.sort_values(by = ['Zahl'], ascending = False, inplace = True)
df_dict_tweetskb_domaene.head(10)

,Domain,Zahl,prozent
2,0,8024014,549.446993
14,twitter.com,259980,17.802216
28,youtu.be,63213,4.328531
37,instagram.com,50250,3.440885
348,twittascope.com,37526,2.569605
31,fllwrs.com,32446,2.221751
65,youtube.com,25874,1.771731
77,onlyfans.com,17436,1.193936
80,twitch.tv,14778,1.011928
62,facebook.com,14721,1.008025


In [12]:
#Die Ergebnisse speichern.
pfad_tsv = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/Tweetskb_stichprobe_domaene.tsv'

with open(pfad_tsv, 'w', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        for idx, row in df_dict_tweetskb_domaene.iterrows():
            tsv_writer.writerow(row.values.tolist())

****

**Die einzigartigen Domäne kategorisieren**

***

In [13]:
#Hier wird mit Hilfe von Pydomain die Domäne kategorisiert
df_tweetskb_domaene = df_dict_tweetskb_domaene[['Domain']]
df_tweetskb_domaaene_shalla_kat = shalla_cat(df_tweetskb_domaene, domain_names='Domain')

Using cached Shallalist data from local (C:\Users\mdsam\.pydomains\shalla_2017.csv.bz2)...
Loading Shallalist data file...


In [14]:
df_tweetskb_domaaene_shalla_kat.head()

,Domain,shalla_2017_domain,shalla_2017_cat
0,0,0,NaN
1,twitter.com,twitter.com,socialnet
2,youtu.be,youtu.be,movies|urlshortener
3,instagram.com,instagram.com,socialnet|imagehosting
4,twittascope.com,twittascope.com,fortunetelling


In [15]:
#Erste Zeile wird gelöscht
df_tweetskb_domaaene_shalla_kat.drop(index=df_tweetskb_domaaene_shalla_kat.index[:1], axis=0, inplace=True)

In [16]:
#Eine Spalte der Dataframe wird gelöscht.
df_tweetskb_domaaene_shalla_kat = df_tweetskb_domaaene_shalla_kat[['Domain', 'shalla_2017_cat']]
df_tweetskb_domaaene_shalla_kat.head()

,Domain,shalla_2017_cat
1,twitter.com,socialnet
2,youtu.be,movies|urlshortener
3,instagram.com,socialnet|imagehosting
4,twittascope.com,fortunetelling
5,fllwrs.com,NaN


In [17]:
#Hier wird mithilfe von get_cat() die erste Kategorie für die Domain ausgewäht, die zu mehrere Kategorien zugewiesen wurde.
def get_cat(domains):
    l = domains.split('|')
    return l[0]

df_tweetskb_domaaene_shalla_kat['shalla_2017_cat'] = df_tweetskb_domaaene_shalla_kat['shalla_2017_cat'].astype(str)

df_tweetskb_domaaene_shalla_kat['dom_cat'] = df_tweetskb_domaaene_shalla_kat.apply(lambda row: get_cat(row['shalla_2017_cat']), axis=1)
print('Domains Kategorisiert!')


Domains Kategorisiert!


In [18]:
df_tweetskb_domaaene_shalla_kat.head()

,Domain,shalla_2017_cat,dom_cat
1,twitter.com,socialnet,socialnet
2,youtu.be,movies|urlshortener,movies
3,instagram.com,socialnet|imagehosting,socialnet
4,twittascope.com,fortunetelling,fortunetelling
5,fllwrs.com,nan,nan


In [19]:
#Die Ergebnisse speichern.
pfad_tsv = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/Tweetskb_stichprobe_domaene_kategorisiert.tsv'

with open(pfad_tsv, 'w', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        for idx, row in df_tweetskb_domaaene_shalla_kat.iterrows():
            tsv_writer.writerow(row.values.tolist())

****

**Alle Domäne in Tweets mithilfe der Domain-Kategorie-Liste kategorisieren** 

***

In [23]:
#Hir wird jede Zeile der Dataframe(jeder Tweet) gelesen, die benutzten Domäne werden zu get_dom_categorisiert() geschickt.
#Für jede Domain in Tweet such die Methode die Liste.
#Falls die Domain kategorisiert ist, nimmt sie die Kategorie der Domain.
#Alle benutzten Kategorien werden mit einer Leerzeichen dazwischen zusammengefügt.
#Das Endergebnis wird zurückgegeben.

def get_dom_categorisiert(domains):
    dom = ''
    list_dom = domains.split()
    for i in range(0, len(list_dom)):
        line = df_tweetskb_domaaene_shalla_kat.loc[df_tweetskb_domaaene_shalla_kat['Domain'] == list_dom[i]]
        line = line.values.tolist()
        #print(list_dom[i], line)
        try :
            if i == 0 :
                dom = dom + line[0][2]
            else:
                dom = dom + ' ' +line[0][2]
        except IndexError:
            print('Index out of range!')
            #print(line)
            
    return dom

df_tweetskb['Domäne'] = df_tweetskb['Domäne'].astype(str)

df_tweetskb['dom_cat'] = df_tweetskb.apply(lambda row: get_dom_categorisiert(row['Domäne']) if (row['Domäne'] != '0') else 0, axis=1)
print('Domains!!!')



Domains!!!


In [27]:
#Die Features der Dataframe einschränken
df_tweetskb = df_tweetskb[['Tweet_Id', 'Ist_gelöscht', 'Domäne', 'Anzahl der URLs', 'dom_cat']]
df_tweetskb.head(10)

,Tweet_Id,Ist_gelöscht,Domäne,Anzahl der URLs,dom_cat
0,1178791636039589889,False,dcuniverseinfinite.com,1,nan
1,1178791641265639424,True,carrd.co,1,nan
2,1178791640573579264,True,0,0,0
3,1178791644797255680,True,0,0,0
4,1178791648056217600,False,phenomhoopreport.com,1,nan
5,1178791648962236416,False,0,0,0
6,1178791657380044800,False,0,0,0
7,1178791661536722949,True,airtime.pro airtime.pro,2,nan nan
8,1178791662841159681,False,mobihealthnews.com,1,nan
9,1178791665735208960,False,dailytelegraph.com.au,1,news


In [28]:
#Daten speichern
pfad='C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/urls/predicted/tweetskb_domains_shalla_categorisieret.tsv'

with open(pfad, 'w', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        for idx, row in df_tweetskb.iterrows():
            tsv_writer.writerow(row.values.tolist())

***

**Die Anzahl der benutzten Domäne, die zu 10 meist benutzten Kategorien gehören, berechnen**

***

In [17]:
#Hier werden die Tweets mit kategorisierten Domäne gelesen. 
pfad ='C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/urls/predicted/shalla/tweetskb_domains_shalla_categorisieret.tsv'

df_tweetskb_domaene_shalla_cat = pd.read_csv(pfad, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Ist_gelöscht', 'Domäne', 'Anzahl der Domäne', 'dom_cat'])
print(df_tweetskb_domaene_shalla_cat.shape)
df_tweetskb_domaene_shalla_cat.head()

(9409841, 5)


,Tweet_Id,Ist_gelöscht,Domäne,Anzahl der Domäne,dom_cat
0,1178791636039589889,False,dcuniverseinfinite.com,1,NaN
1,1178791641265639424,True,carrd.co,1,NaN
2,1178791640573579264,True,0,0,0
3,1178791644797255680,True,0,0,0
4,1178791648056217600,False,phenomhoopreport.com,1,NaN


In [20]:
#Hier werden die meist benutzten Kategorien in tweetsKB berechnet.
df_tweetskb_domaene_shalla_cat['dom_cat'] = df_tweetskb_domaene_shalla_cat['dom_cat'].astype(str)

dict_tweetskb_domaene_cat = {}
x = 0
list_cat = []
for idx, row in df_tweetskb_domains_shalla_cat.iterrows():
    list_cat = (row.values.tolist()[4])
    list_cat = str(list_cat).split()
    for i in range(0, len(list_cat)):
        cat = list_cat[i]
        x += 1
        if cat not in dict_tweetskb_domaene_cat:
            dict_tweetskb_domaene_cat[cat] = 1
        else :
            dict_tweetskb_domaene_cat[cat] = dict_tweetskb_domaene_cat[cat] +1
    #if x == 50:
    #    break

In [21]:
#Eine Dataframe aus die Dictionery bilden
df_dict_tweetskb_dom_cat = pd.DataFrame(dict_tweetskb_domaene_cat.items(), columns=['Domain_kat','Zahl'])

In [22]:
print('Anzahl der Kategorien:' ,df_dict_tweetskb_dom_cat.Domain_kat.size)

Anzahl der Kategorien: 76


In [25]:
#Anzahl der Domäne(URLs) in TweetsKB berechnen
domaene_sum = df_tweetskb_domains_shalla_cat['Anzahl der Domäne'].sum()
domaene_sum

1460380

In [26]:
#Prozentualen Anteil der beneutzten Kategorien bestimmen
df_dict_tweetskb_dom_cat['prozent'] =  df_dict_tweetskb_dom_cat['Zahl'].apply(lambda row : ((row /domaene_sum) * 100) )

In [28]:
#Dataframe aufsteigend sortieren.
#Dataframe neu indexieren und einen Teil davon ausgeben.
df_dict_tweetskb_dom_cat['Zahl'] = df_dict_tweetskb_dom_cat['Zahl'].astype(int) 
df_dict_tweetskb_dom_cat.sort_values(by = ['Zahl'], ascending = False, inplace = True)
df_dict_tweetskb_dom_cat = df_dict_tweetskb_dom_cat.reset_index()
df_dict_tweetskb_dom_cat.head(15)

,index,Domain_kat,Zahl,prozent
0,1,0,8024014,549.446993
1,0,nan,687141,47.052206
2,3,socialnet,333322,22.824333
3,2,news,138386,9.476027
4,6,movies,94252,6.453937
5,23,fortunetelling,37554,2.571522
6,5,porn,33848,2.317753
7,9,music,19686,1.348005
8,12,hobby/games-misc,17182,1.176543
9,14,urlshortener,16777,1.148811


In [24]:
#Die Anzahl der benutzten Domäne, die zu Kategorie soziale Netzwerke gehören wird hier berechnet
def get_socialnet_num(dom_cat):
    l = dom_cat.split()
    num = l.count('socialnet')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetskb_domains_shalla_cat['dom_cat'] = df_tweetskb_domains_shalla_cat['dom_cat'].astype(str) 

df_tweetskb_domains_shalla_cat['socialnet'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_socialnet_num(row['dom_cat']), axis=1)
print('socialnet!')

socialnet!


In [25]:
#Die Anzahl der benutzten Domäne, die zu Kategorie News gehören wird hier berechnet
def get_news_num(dom_cat):
    l = dom_cat.split()
    num = l.count('news')
    return float( "{:.2f}".format((num / len(l) )) )


df_tweetskb_domains_shalla_cat['news'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_news_num(row['dom_cat']), axis=1)

In [26]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Movies gehören wird hier berechnet
def get_movies_num(dom_cat):
    l = dom_cat.split()
    num = l.count('movies')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetskb_domains_shalla_cat['movies'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_movies_num(row['dom_cat']), axis=1)
print('movies!')

movies!


In [27]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Fortunetelling gehören wird hier berechnet
def get_fortunetelling_num(dom_cat):
    l = dom_cat.split()
    num = l.count('fortunetelling')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetskb_domains_shalla_cat['fortunetelling'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_fortunetelling_num(row['dom_cat']), axis=1)
print('fortunetelling!')

fortunetelling!


In [28]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Porn gehören wird hier berechnet

def get_porn_num(dom_cat):
    l = dom_cat.split()
    num = l.count('porn')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetskb_domains_shalla_cat['porn'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_porn_num(row['dom_cat']), axis=1)
print('porn!')

porn!


In [29]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Music gehören wird hier berechnet
def get_music_num(dom_cat):
    l = dom_cat.split()
    num = l.count('music')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetskb_domains_shalla_cat['music'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_music_num(row['dom_cat']), axis=1)
print('music!')

music!


In [30]:
#Die Anzahl der benutzten Domäne, die zu Kategorie hobby/games_misc gehören wird hier berechnet
def get_hobby_num(dom_cat):
    l = dom_cat.split()
    num = l.count('hobby/games-misc')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetskb_domains_shalla_cat['hobby/games_misc'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_hobby_num(row['dom_cat']), axis=1)
print('hobby/games-misc!')

hobby/games-misc!


In [31]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Shopping gehören wird hier berechnet
def get_shopping_num(dom_cat):
    l = dom_cat.split()
    num = l.count('shopping')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetskb_domains_shalla_cat['shopping'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_shopping_num(row['dom_cat']), axis=1)
print('shopping!')

shopping!


In [32]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Forum gehören wird hier berechnet
def get_forum_num(dom_cat):
    l = dom_cat.split()
    num = l.count('forum')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetskb_domains_shalla_cat['forum'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_forum_num(row['dom_cat']), axis=1)
print('forum!')

forum!


In [33]:
#Die Anzahl der benutzten Domäne, die zu Kategorie Searchengines gehören wird hier berechnet
def get_searchengines_num(dom_cat):
    l = dom_cat.split()
    num = l.count('searchengines')
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetskb_domains_shalla_cat['searchengines'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_searchengines_num(row['dom_cat']), axis=1)
print('searchengines!')

searchengines!


In [34]:
#Die Anzahl der benutzten Domäne, die alles außer bis jetzt bestimmten Kategorien sind, wird hier berechnet
def get_others_num(dom_cat):
    l = dom_cat.split()
    num = l.count('socialnet') + l.count('news') + l.count('movies') + l.count('fortunetelling') + l.count('porn') + l.count('music') + l.count('hobby/games_misc') + l.count('shopping') + l.count('forum') + l.count('searchengines') + l.count('0') + l.count('nan')
    num =  len(l) - num
    return float( "{:.2f}".format((num / len(l) )) )

df_tweetskb_domains_shalla_cat['others'] = df_tweetskb_domains_shalla_cat.apply(lambda row: get_others_num(row['dom_cat']), axis=1)
print('others!')

others!


In [35]:
df_tweetskb_domains_shalla_cat.head(10)

,Tweet_Id,is_deleted,URLs_dom,URLs_num,dom_cat,socialnet,news,movies,fortunetelling,porn,music,hobby/games_misc,shopping,forum,searchengines,others
0,1178791636039589889,False,dcuniverseinfinite.com,1,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1178791641265639424,True,carrd.co,1,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1178791640573579264,True,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1178791644797255680,True,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1178791648056217600,False,phenomhoopreport.com,1,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1178791648962236416,False,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1178791657380044800,False,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1178791661536722949,True,airtime.pro airtime.pro,2,nan nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1178791662841159681,False,mobihealthnews.com,1,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1178791665735208960,False,dailytelegraph.com.au,1,news,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
#Die Ergebnisse wird hier berechnet.
pfad2='C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/urls/predicted/shalla/tweetscov19_domains_shalla_categorisieret_top_10.tsv'

with open(pfad2, 'w', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        for idx, row in df_tweetscov19_domains_shalla_cat.iterrows():
            tsv_writer.writerow(row.values.tolist())